In [ ]:
import pandas
import numpy as np

import climb_detector as climb_detector
import get_locations as locator
import parser as parser
import plotter as plotter
import util as util

In [ ]:
name = "Étape_du_tour_femmes"
route = parser.open_gpx(rf"C:\Users\Thibaut\Documents\Python Scripts\Cycling\traces\{name}.gpx")

df = parser.parse_gpx(route)
route.df = parser.apply_slope_smoothing(route.df, target_meters=100)
stats = parser.compute_stats_gpx(route)

params = {
    "max_pause_length_m": 200,
    "max_pause_descent_m": 10,
    "start_threshold_slope": 2.0,
}

climbs_df = climb_detector.detect_significant_segments(route.df, kind="climb", **params)
descents_df = climb_detector.detect_significant_segments(route.df, kind="descent", **params)

route.descents_df = descents_df
route.climbs_df = climbs_df

df_with_locations = locator.add_location(route)

In [ ]:
stats

In [ ]:
print(route.df["grade"].max())
print(route.df["grade"].min())

In [ ]:
params = {
    "max_pause_length_m": 200,
    "max_pause_descent_m": 10,
    "start_threshold_slope": 2.0,
}

climbs_df = climb_detector.detect_significant_segments(route.df, kind="climb", **params)
descents_df = climb_detector.detect_significant_segments(route.df, kind="descent", **params)

route.descents_df = descents_df
route.climbs_df = climbs_df

In [ ]:
import folium
import folium.plugins as plugins
import leafmap.foliumap as leafmap

import util as util
from route import GPXData

def create_route_map(
    route: GPXData,
    tile_style: str = "OpenStreetMap",
    color_by_slope: bool = True,
) -> None:

    df = route.df
    climbs_df = route.climbs_df
    descents_df = route.descents_df
    coords = df[["lat", "lon"]].values.tolist()

    # Map center and bounds
    center = coords[len(coords) // 2]
    m = leafmap.Map(location=center, zoom_start=13, control_scale=True, tiles=None)

    legend_dict = {
        "-10% and below": "#4682B4",      # steel blue
        "-10% to -4%": "#87CEEB",        # sky blue
        "-4% to -2%": "#ADD8E6",         # light blue
        "-2% to 2%": "#FFFFFF",          # white
        "2% to 4%": "#75f60c",           # green
        "4% to 6%": "#00a0ff",           # blue
        "6% to 8%": "#ffd300",           # yellow
        "8% to 10%": "#ee0000",          # red
        "10% to 12%": "#800080",         # purple
        "12% and above": "#444444",      # black
    }

    m.add_legend(title="Slope Grade Legend", legend_dict=legend_dict)

    folium.TileLayer(tiles=tile_style, name=tile_style, opacity=0.8).add_to(m)

    # Segment coloring
    for i in range(1, len(coords)):
        segment = [coords[i - 1], coords[i]]
        color = util.getcolor(df["plot_grade"].iloc[i]) if color_by_slope else "#999999"
        folium.PolyLine(segment, color=color, weight=4, opacity=1).add_to(m)

    # Start and end markers
    folium.Marker(
        coords[0], icon=folium.Icon(color="green", icon="play"), popup="Start", 
    ).add_to(m)
    folium.Marker(
        coords[-1], icon=folium.Icon(color="red", icon="stop"), popup="End"
    ).add_to(m)

    # Climb markers
    if climbs_df is not None and not climbs_df.empty:
        for idx, row in climbs_df.iterrows():
            start_idx = (row["start_idx"])
            lat, lon = df.loc[start_idx, ["lat", "lon"]]
            folium.Marker(
                location=[lat, lon],
                popup=f"Climb {idx + 1}: {int(row['elev_gain'])}m ↑, L={int(row["length_m"]/1000)} km, slope={np.round(row["avg_slope"],1)} %",
                # icon=folium.DivIcon(
                #     html=f"<div style='font-size: 18px; color: red;'>{idx + 1}</div>"
                # ),
                icon = plugins.BeautifyIcon(
                     icon="mountain",
                     icon_shape="circle",
                     border_color='purple',
                     text_color="#007799",
                     background_color='white'
                 )
            ).add_to(m)

    folium.LayerControl().add_to(m)

    # Fit to bounds
    sw = df[["lat", "lon"]].min().values.tolist()
    ne = df[["lat", "lon"]].max().values.tolist()
    m.fit_bounds([sw, ne])

    return m


In [ ]:
m = plotter.create_route_map(route, color_by_slope=True)
m.save("Map1.html")


In [ ]:
import get_locations as locator

df_with_locations = locator.add_location(route)

In [ ]:
import matplotlib.pyplot as plt


df = route.df
climbs_df = route.climbs_df
descents_df = route.descents_df

annotationsAnchor = df['ele'].max() * 1.1
style_city = dict(size=10, color='grey')
style_slope = dict(size=10, color='black')
style_max = dict(size=10, color='red')

fig, ax = plt.subplots(figsize=(12, 4))
ax.set_xlabel("Kilometers")
ax.set_ylabel("Elevation (m)")
ax.spines[['right', 'top']].set_visible(False)

ax.plot(df["distance"] / 1000, df["ele"], color="#999999", linewidth=1.5, alpha=0.7)

old_index = 0
for index in df[df["location"].notnull()].index:
    #ensure that two annotations do not overlap
    
    if index > 0 and abs(df["distance"].values[index] - df["distance"].values[old_index])/1000 < 2:
        continue

    ax.annotate(
        df["location"].values[index],
        xy=(df["distance"].values[index] / 1000, df["ele"].values[index]),
        xytext=(df["distance"].values[index] / 1000, annotationsAnchor),
        arrowprops=dict(arrowstyle="-", color='lightgray'),
        horizontalalignment='center',
        rotation=90,
        **style_city
    )
    old_index = index

old_x_align = 0 
y_align = 0.2

for segment_df, color in [(climbs_df, "#FFA500")]:
    if segment_df is not None:
        for _, row in segment_df.iterrows():
            segment = df[
                (df["distance"] / 1000 >= row["start_km"])
                & (df["distance"] / 1000 <= row["end_km"])
            ]
            ax.fill_between(
                segment["distance"] / 1000, segment["ele"], color=color, alpha=0.4
            )
            
            ax.annotate(
                f"{round(segment['ele'].values[-1])}m",
                xy=(segment["distance"].values[-1] / 1000, segment["ele"].values[-1]),
                xytext=(segment["distance"].values[-1] / 1000, 1.02 * segment["ele"].values[-1]),
                horizontalalignment='center',
                rotation=45,
                **style_max
            )

            mean_slope = round(row["avg_slope"],1)
            start = row["start_km"]
            end = row["end_km"] 

            x_align = (start+end)/2
            if x_align - old_x_align < 5:
                y_align = y_align + 100
    
            ax.annotate(
                f"{mean_slope}%",
                xy=( x_align, y_align),
                xytext=( x_align, y_align),
                horizontalalignment='center',
                rotation=0,
                **style_slope
            )
            y_align = 0.2
            old_x_align = x_align

In [ ]:
fig.savefig("aa.png")

In [ ]:
# plotter.plot_elevation_colored_by_slope(route, simplified=True)

In [ ]:
# plotter.plot_elevation_colored_by_slope(route, simplified=False, color_mode="Detailed Slope")

In [ ]:
# df = route.df
# climbs_df = route.climbs_df
# descents_df = route.descents_df

# annotationsAnchor = df['ele'].max() * 1.1
# style_city = dict(size=10, color='grey')

# fig, ax = plt.subplots(figsize=(12, 4))
# ax.set_xlabel("Kilometers")
# ax.set_ylabel("Elevation (m)")
# ax.spines[['right', 'top']].set_visible(False)

# ax.plot(df["distance"] / 1000, df["ele"], color="#999999", linewidth=1.5, alpha=0.7)

# old_index = 0
# for index in df[df["location"].notnull()].index:
#     #ensure that two annotations do not overlap
    
#     if index > 0 and abs(df["distance"].values[index] - df["distance"].values[old_index])/1000 < 2:
#         continue

#     ax.annotate(
#         df["location"].values[index],
#         xy=(df["distance"].values[index] / 1000, df["ele"].values[index]),
#         xytext=(df["distance"].values[index] / 1000, annotationsAnchor),
#         arrowprops=dict(arrowstyle="-", color='lightgray'),
#         horizontalalignment='center',
#         rotation=90,
#         **style_city
#     )
#     old_index = index

#     style_slope = dict(size=10, color='black')
#     style_max = dict(size=10, color='red')

#     # for segment_df in [climbs_df]:
#     #     if segment_df is not None:
#     #         for _, row in segment_df.iterrows():
#     #             segment = df[
#     #                 (df["distance"] / 1000 >= row["start_km"])
#     #                 & (df["distance"] / 1000 <= row["end_km"])
#     #             ]
                
#     #             ax.annotate(
#     #                 f"{round(segment['ele'].values[-1])}m",
#     #                 xy=(segment["distance"].values[-1] / 1000, segment["ele"].values[-1]),
#     #                 xytext=(segment["distance"].values[-1] / 1000, 1.02 * segment["ele"].values[-1]),
#     #                 horizontalalignment='center',
#     #                 rotation=45,
#     #                 **style_max
#     #             )

#     #             mean_slope = round(row["avg_slope"],1)
#     #             start = row["start_km"]
#     #             end = row["end_km"] 

#     #             x_align = (start+end)/2
#     #             if x_align - old_x_align < 5:
#     #                 y_align = y_align + 100
        
#     #             ax.annotate(
#     #                 f"{mean_slope}%",
#     #                 xy=( x_align, y_align),
#     #                 xytext=( x_align, y_align),
#     #                 horizontalalignment='center',
#     #                 rotation=0,
#     #                 **style_slope
#     #             )
#     #             y_align = 0.2
#     #             old_x_align = x_align

#     # if color_mode == "Detailed Slope":
#         # Couleur pour chaque pente calculée
#     for i in range(1, len(df)):
#         x = df["distance"].iloc[i - 1 : i + 1] / 1000
#         y = df["ele"].iloc[i - 1 : i + 1]
#         color = util.getcolor(df["plot_grade"].iloc[i])
#         ax.fill_between(x, 0, y, color=color, alpha=0.8)


In [ ]:
df

In [ ]:
climbs_df

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.ndimage import uniform_filter1d

# -----------------------------
# Example DataFrames
# df: distance in meters, ele in meters, grade in %
# climbs_df: type, start_km, end_km, elev_gain, length_m, avg_slope, start_idx, end_idx
# -----------------------------
# Parameters
window_m = 100

def compute_avg_slope(df, window_m=100):
    median_dx = df['distance'].diff().median()
    if np.isnan(median_dx) or median_dx <= 0:
        median_dx = 20  # fallback
    window_points = max(int(window_m / median_dx), 1)
    
    df = df.copy()
    df['avg_slope'] = df['grade'].rolling(window=window_points, min_periods=1).mean()
    df['smoothed_ele'] = uniform_filter1d(df['ele'], size=window_points)
    df['color'] = df['avg_slope'].apply(util.getcolor)
    return df

df = compute_avg_slope(df, window_m)

# -----------------------------
# Create figure
# -----------------------------
fig = go.Figure()

# Plot background elevation as a thin grey line
fig.add_trace(go.Scatter(
    x=df['distance']/1000,
    y=df['smoothed_ele'],
    mode='lines',
    line=dict(color='grey', width=2),
    name='Elevation',
    hoverinfo='skip'
))

# Plot filled slope segments
start_idx = 0
for i in range(1, len(df)):
    if df['color'].iloc[i] != df['color'].iloc[start_idx] or i == len(df)-1:
        color = df['color'].iloc[start_idx]
        if color:  # only plot colored segments
            seg_x = df['distance'].iloc[start_idx:i+1]/1000
            seg_y = df['smoothed_ele'].iloc[start_idx:i+1]
            slope_length = (seg_x.iloc[-1] - seg_x.iloc[0])
            fig.add_trace(go.Scatter(
                x=seg_x,
                y=seg_y,
                fill='tozeroy',
                mode='none',
                fillcolor=color,
                hovertemplate=(
                    "Distance: %{x:.2f} km<br>"
                    "Elevation: %{y:.0f} m<br>"
                    f"Length: {slope_length:.2f} km<br>"
                    f"Avg slope: {df['avg_slope'].iloc[start_idx]:.1f}%"
                ),
                name=f"Slope {df['avg_slope'].iloc[start_idx]:.1f}%"
            ))
        start_idx = i



# -----------------------------
# Annotate climbs
# -----------------------------
last_annot_x = -10
min_spacing = 8.5
y_base = 0#df['smoothed_ele'].min() - 0.02 * (df['smoothed_ele'].max() - df['smoothed_ele'].min())

for i, row in climbs_df.iterrows():
    x = (row['start_km'] + row["end_km"])/2

    if row['length_m']/1000 >= 4:
        fig.add_annotation(
            x=x,
            y=y_base ,
            text=f"{row['length_m']/1000:.2f} km<br>{round(row['avg_slope'],1)}%",  # <br> for line break in Plotly
            showarrow=False,
            yanchor="bottom",
            xanchor="center",
            font=dict(size=12, color="black"),
            bgcolor="white",
            bordercolor="black",
            borderwidth=1,
            borderpad=2
    )
annotationsAnchor = df['ele'].max() * 1.05
old_index = None

for index in df[df["location"].notnull()].index:
    loc_name = df["location"].iloc[index]
    # Skip if too close to previous annotation
    if old_index is not None and abs(df["distance"].iloc[index] - df["distance"].iloc[old_index]) / 1000 < 2:
        continue

    x_pos = df["distance"].iloc[index] / 1000
    y_elev = df["ele"].iloc[index]

    # Add dashed vertical line from elevation to text
    fig.add_shape(
        type="line",
        x0=x_pos, y0=y_elev,
        x1=x_pos, y1=annotationsAnchor,
        line=dict(color="gray", width=1, dash="dot"),
        layer="below"
    )

    # Add vertical text label, aligned at annotationsAnchor
    fig.add_annotation(
        x=x_pos,
        y=annotationsAnchor,
        text=loc_name,
        showarrow=False,
        textangle=-90,
        xanchor="center",
        yanchor="bottom",
        font=dict(size=12, color='black')
    )
    old_index = index


# -----------------------------
# Layout
# -----------------------------
fig.update_layout(
    title="Elevation Profile",
    xaxis_title="Distance (km)",
    yaxis_title="Elevation (m)",
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=True),
    plot_bgcolor='rgba(200,200,200,0.2)',
    hovermode="closest",
    showlegend=False
)

fig.update_yaxes(
    title="Elevation",
    ticksuffix=" m",      # adds " m" after each tick
    showgrid=False
)

fig.update_xaxes(
    title="Distance",
    ticksuffix=" km",     # adds " km" after each tick
    showgrid=False
)

fig.update_yaxes(range=[0, 1.1*df["ele"].max()])

fig.update_layout(
    height=600  # default is ~450
)

fig.show()

In [92]:
climbs_df

,type,start_km,end_km,elev_gain,length_m,avg_slope,start_idx,end_idx
0,climb,4.599101,17.704535,859.8,13105.434396,6.560637,58,455
1,climb,30.428850,31.492840,51.4,1063.990430,4.830871,669,694
2,climb,32.063047,34.328683,87.1,2265.636249,3.844395,700,746
3,climb,37.750256,40.024973,51.1,2274.716927,2.246433,809,838
4,climb,43.179909,48.677783,191.0,5497.874013,3.474070,870,971
5,climb,59.631368,61.028046,91.4,1396.677924,6.544100,1182,1227
6,climb,73.031560,77.829262,283.5,4797.702612,5.909078,1356,1493
7,climb,86.771914,88.555291,84.8,1783.376572,4.755025,1628,1669
8,climb,95.319334,115.850842,1549.8,20531.508156,7.548398,1752,2477


In [99]:
import plotly.graph_objects as go
import pandas as pd

# Example: your climbs_df
climbs_df = pd.DataFrame({
    "type": ["climb"]*9,
    "start_km": [4.599101, 30.428850, 32.063047, 37.750256, 43.179909, 59.631368, 73.031560, 86.771914, 95.319334],
    "end_km":   [17.704535, 31.492840, 34.328683, 40.024973, 48.677783, 61.028046, 77.829262, 88.555291, 115.850842],
    "elev_gain": [859.8, 51.4, 87.1, 51.1, 191.0, 91.4, 283.5, 84.8, 1549.8],
    "length_m": [13105.43, 1063.99, 2265.63, 2274.72, 5497.87, 1396.68, 4797.70, 1783.38, 20531.51],
    "avg_slope": [6.56, 4.83, 3.84, 2.25, 3.47, 6.54, 5.91, 4.75, 7.55],
    "start_idx": [58, 669, 700, 809, 870, 1182, 1356, 1628, 1752],
    "end_idx":   [455, 694, 746, 838, 971, 1227, 1493, 1669, 2477]
})

# Suppose df is your full elevation profile DataFrame with columns:
# 'km', 'ele', 'avg_slope_100m', 'location'...
# We'll reuse your getcolor but skip coloring for small negative slopes.
def getcolor(grade: float) -> str:
    if grade < -10:
        return "#A0522D"  # brown for steep descents
    elif grade < 2 and grade >= -10:
        return "#F5F5DC"  # beige for flat to mild descents
    elif 2 <= grade < 4:
        return "#75f60c"
    elif 4 <= grade < 6:
        return "#00a0ff"
    elif 6 <= grade < 8:
        return "#ffd300"
    elif 8 <= grade < 10:
        return "#ee0000"
    elif 10 <= grade < 12:
        return "#800080"
    elif grade >= 12:
        return "#444444"
    return "#FFFFFF"

# Compute global scale
global_min_x = df['distance'].min()
global_max_x = df['distance'].max()
global_min_y = 0
global_max_y = df['ele'].max() * 1.15  # 15% headroom

# Stats container
stats_list = []

# Loop over climbs
for i, climb in climbs_df.iterrows():
    seg_df = df.iloc[climb.start_idx:climb.end_idx+1].copy()
    colors = [getcolor(g) for g in seg_df['avg_slope_100m']]

    # Compute stats
    length_km = climb.length_m / 1000
    gain_m = climb.elev_gain
    avg_slope = climb.avg_slope
    max_slope_100m = seg_df['avg_slope_100m'].max()
    stats_list.append({
        "Climb": i+1,
        "Length (km)": round(length_km, 2),
        "Gain (m)": round(gain_m, 1),
        "Avg slope (%)": round(avg_slope, 2),
        "Max slope 100 m (%)": round(max_slope_100m, 1)
    })

    # Build figure
    fig = go.Figure()

    start_idx = 0
    for j in range(1, len(seg_df)):
        if colors[j] != colors[j-1] or j == len(seg_df)-1:
            seg_x = seg_df['distance'].iloc[start_idx:j+1]
            seg_y = seg_df['ele'].iloc[start_idx:j+1]
            avg_slope_val = seg_df['avg_slope_100m'].iloc[start_idx]
            length_val = (seg_x.iloc[-1] - seg_x.iloc[0]) * 1000
            fig.add_trace(go.Scatter(
                x=seg_x,
                y=seg_y,
                fill='tozeroy',
                mode='none',
                fillcolor=colors[start_idx],
                hovertemplate=(
                    f"Avg slope (100m): {avg_slope_val:.1f}%<br>"
                    f"Length: {length_val/1000:.2f} km"
                ),
                name=f"Slope {avg_slope_val:.1f}%"
            ))
            start_idx = j

    # Same scale for all climbs
    # fig.update_yaxes(range=[global_min_y, global_max_y], ticksuffix=" m")
    # fig.update_xaxes(range=[global_min_x, global_max_x], ticksuffix=" km")

    fig.update_layout(
        height=650,
        plot_bgcolor="white",
        xaxis=dict(showgrid=False),
        yaxis=dict(showgrid=False),
        title=f"Climb {i+1} — {length_km:.2f} km | {gain_m:.0f} m gain | {avg_slope:.1f}% avg",
        margin=dict(t=80, b=50, l=50, r=50)
    )

    fig.show()

# Display stats as DataFrame
stats_df = pd.DataFrame(stats_list)
print(stats_df)


   Climb  Length (km)  Gain (m)  Avg slope (%)  Max slope 100 m (%)
0      1        13.11     859.8           6.56                 12.1
1      2         1.06      51.4           4.83                  7.8
2      3         2.27      87.1           3.84                  6.6
3      4         2.27      51.1           2.25                  4.1
4      5         5.50     191.0           3.47                  8.7
5      6         1.40      91.4           6.54                 12.2
6      7         4.80     283.5           5.91                  8.8
7      8         1.78      84.8           4.75                  7.9
8      9        20.53    1549.8           7.55                 14.6


In [94]:
seg_df

,lat,lon,ele,distance,grade,plot_grade,location,avg_slope_100m,color,avg_slope,smoothed_ele
58,45.578657,5.966981,310.3,4599.101014,1.084607,1.443463,Saint-Alban-Leysse,1.104357,#FFFFFF,1.104357,309.70
59,45.578881,5.967259,311.0,4632.093810,2.121675,3.708166,None,1.603141,#FFFFFF,1.603141,310.65
60,45.579151,5.967879,315.5,4688.924789,7.918216,7.204287,None,5.019946,#00a0ff,5.019946,313.25
61,45.579362,5.968067,318.7,4716.575431,11.572968,9.053860,None,9.745592,#ee0000,9.745592,317.10
62,45.579583,5.968179,320.7,4742.649703,7.670396,9.569605,None,9.621682,#ee0000,9.621682,319.70
...,...,...,...,...,...,...,...,...,...,...,...
451,45.644572,6.015228,1168.1,17562.216665,4.913761,4.283863,None,5.572992,#00a0ff,5.572992,1166.75
452,45.644833,6.015702,1168.9,17609.120843,1.705605,3.065505,None,3.309683,#75f60c,3.309683,1168.50
453,45.645038,6.015892,1169.6,17636.282659,2.577147,2.152641,None,2.141376,#75f60c,2.141376,1169.25
454,45.645287,6.016103,1170.3,17668.464027,2.175172,2.046153,None,2.376159,#75f60c,2.376159,1169.95


In [ ]:
row